In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pprint

1) Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта job.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе:

Наименование вакансии,
Предлагаемую зарплату,

Ссылку на саму вакансию

2) Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)

In [2]:
#Посмотрим на первоначальную ссылку:
initial_link = 'https://novosibirsk.hh.ru/search/vacancy?text=developer&search_field=description&salary=&currency_code=RUR&only_with_salary=true&\
experience=doesNotMatter&order_by=relevance&search_period=&items_on_page=100&no_magic=true'

In [3]:
#Зададим поиск страницы функцией:

def get_page(position, page):
    api = 'https://novosibirsk.hh.ru/search/vacancy'
    #Без этого параметра не получилось вытащить данные
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko)' \
                 'Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}
    params = {
        "text": '"' + position + '"',
        "search_field": "name",
        "items_on_page": 100,
        "only_with_salary": "true",
        "page": page
    }

    html = requests.get(api, params=params, headers=headers).text
    parsed_html = bs(html, 'lxml')
    return parsed_html

In [5]:
#Выберем весь блок с вакансиями

def get_all_data(html):
    whole_list = html.findAll("div", {"data-qa": "vacancy-serp__vacancy"})
    return whole_list


In [6]:
#Пагинация
def get_page_number(html):
    pages = html.findAll("span", {"class": "bloko-button-group"})
    page_count = pages[0].findAll('', {"data-qa": "pager-page"})
    return len(page_count)

In [7]:
# Перебирам вакансии, вытаскиваем нужные данные:

def get_vacation_data(position_html):
    position_data = position_html.find('a', {"data-qa": "vacancy-serp__vacancy-title"})
    title = position_data.text
    link = position_data['href']
    empl_data = position_html.find('a', {"data-qa": "vacancy-serp__vacancy-employer"})
    empl_title = empl_data.text
    empl_url = empl_data['href']
    try:
        salary = position_html.find('div', {"data-qa": "vacancy-serp__vacancy-compensation"}).text
    except AttributeError:
        salary = ''

    vacation = {
        'title': title, 
        'link': link,
        'employer': empl_title, 
        'employer_link': f'https://novosibirsk.hh.ru{empl_url}',
        'salary': salary
        
    }

    return vacation


In [9]:
#Собираем функции вместе
def get_vacations(position):
    clean_positions_list = []

    first_page = get_page(position, 0)

    positions = get_all_data(first_page)

    pages_count = get_page_number(first_page)

    if pages_count > 1:
        for i in range(1, pages_count):
            html = get_page(position, i)
            new_positions = get_all_data(html)
            positions.extend(new_positions)

    for position in positions:
        clean_positions_list.append(get_vacation_data(position))

    return clean_positions_list

In [19]:

get_vacations('developer')

[{'title': 'Java Developers / Angular Developer в Германии',
  'link': 'https://novosibirsk.hh.ru/vacancy/32213365?query=%22developer%22',
  'employer': ' GOD mbH',
  'employer_link': 'https://novosibirsk.hh.ru/employer/3926503',
  'salary': '3\xa0700-5\xa0000 EUR'},
 {'title': 'Senior NodeJS Developer- to Dubai, UAE',
  'link': 'https://novosibirsk.hh.ru/vacancy/32151778?query=%22developer%22',
  'employer': 'ООО Find IT talent',
  'employer_link': 'https://novosibirsk.hh.ru/employer/3486371',
  'salary': '5\xa0000-5\xa0000 USD'},
 {'title': 'Javascript Developer',
  'link': 'https://novosibirsk.hh.ru/vacancy/32062168?query=%22developer%22',
  'employer': ' CIS Force',
  'employer_link': 'https://novosibirsk.hh.ru/employer/3977642',
  'salary': 'от 4\xa0000 EUR'},
 {'title': 'Senior Software Developer (#C++, #Linux)',
  'link': 'https://novosibirsk.hh.ru/vacancy/30964121?query=%22developer%22',
  'employer': 'ООО Знаменка Капитал',
  'employer_link': 'https://novosibirsk.hh.ru/employe